# 11. Constrained Optimization

Many real-world problems have constraints ($g(x) \le 0$). VAMOS handles these natively.

When a problem returns a constraint matrix `G`:
- `G[i, j] <= 0`: Satisfied
- `G[i, j] > 0`: Violated

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from vamos import optimize
from vamos.algorithms import NSGAIIConfig
from vamos.foundation.metrics.pareto import pareto_filter
from vamos.foundation.constraints import ConstraintInfo
from vamos.foundation.problem.constrained import WeldedBeamDesignProblem, CEC2009CF1

plt.style.use("ggplot")
print("Constrained optimization tools loaded!")

## 1. Constraint Analysis

Use `ConstraintInfo` to analyze feasibility of solutions.

In [ ]:
# Example Data
G_example = np.array(
    [
        [-0.5, -0.1],  # Feasible
        [0.1, -0.2],  # Infeasible (violation=0.1)
        [-0.3, 0.5],  # Infeasible (violation=0.5)
        [0.0, 0.0],  # Feasible
    ]
)

info = ConstraintInfo(G_example)
print(f"Feasible Mask: {info.feasible_mask}")
print(f"Total CV: {info.cv}")
print(f"Feasible Count: {info.n_feasible} / {len(info)}")

## 2. Welded Beam Design

A classic constrained engineering problem.
- Minimize Cost & Deflection
- Subject to shear stress, normal stress, etc.

In [ ]:
welded_beam = WeldedBeamDesignProblem()
print(f"Welded Beam: {welded_beam.n_var} vars, {welded_beam.n_constr} constraints")

print("Running Optimization...")
# Unified API automatically handles constrained domination sort
welded_result = optimize(welded_beam, algorithm="nsgaii", pop_size=100, budget=10000, seed=42)

print(f"Found {len(welded_result)} solutions")

In [ ]:
# Analyze & Visualize
if welded_result.G is not None:
    cinfo = ConstraintInfo(welded_result.G)
    print(f"Feasible Solutions: {cinfo.n_feasible} / {len(cinfo)}")

    # Plot
    fig, ax = plt.subplots(figsize=(8, 6))
    F = welded_result.F
    mask = cinfo.feasible_mask

    if mask.any():
        # Filter Pareto optimal subset of feasible solutions
        F_feas = pareto_filter(F[mask])
        if F_feas.size:
            ax.scatter(F_feas[:, 0], F_feas[:, 1], c="green", label="Feasible Pareto")

    if (~mask).any():
        ax.scatter(F[~mask, 0], F[~mask, 1], c="red", marker="x", alpha=0.3, label="Infeasible")

    ax.set_title("Welded Beam Results")
    ax.set_xlabel("Cost")
    ax.set_ylabel("Deflection")
    ax.legend()
    plt.show()

## 3. CEC2009 CF1

A challenging benchmark with disconnected feasible regions.

In [ ]:
cf1 = CEC2009CF1(n_var=10)
print("Running CF1...")

cf1_result = optimize(cf1, algorithm="nsgaii", pop_size=100, budget=5000, seed=42)

if cf1_result.G is not None:
    cinfo = ConstraintInfo(cf1_result.G)
    print(f"CF1 Feasible: {cinfo.n_feasible} / {len(cinfo)}")

    cf1_result.plot(title="CEC2009 CF1 Result")

## 4. Manual Penalty Approach

Demonstrating how to manually implement penalty functions if needed (e.g., for soft constraints).
$$f_i'(x) = f_i(x) + P \cdot CV(x)$$

In [ ]:
class PenalizedWeldedBeam(WeldedBeamDesignProblem):
    def __init__(self, penalty=1000.0):
        super().__init__()
        self.penalty = penalty
        self.n_constr = 0  # Hide constraints from optimizer

    def evaluate(self, X, out, *args, **kwargs):
        temp = {}
        super().evaluate(X, temp, *args, **kwargs)

        # Manual penalty
        cv = np.sum(np.maximum(0, temp["G"]), axis=1)
        out["F"] = temp["F"] + self.penalty * cv[:, None]
        # No G returned -> treated as unconstrained


print("Running Manual Penalty...")
penalized_result = optimize(PenalizedWeldedBeam(penalty=1000), algorithm="nsgaii", pop_size=100, budget=5000, seed=42)

# Verify feasibility on TRUE problem
true_prob = WeldedBeamDesignProblem()
verif_out = {}
true_prob.evaluate(penalized_result.X, verif_out)
cinfo = ConstraintInfo(verif_out["G"])
print(f"Feasible found via Penalty: {cinfo.n_feasible}")

# Plot True Objectives
plt.figure(figsize=(8, 5))
F_true = verif_out["F"]
plt.scatter(F_true[cinfo.feasible_mask, 0], F_true[cinfo.feasible_mask, 1], c="green", label="Feasible")
plt.scatter(F_true[~cinfo.feasible_mask, 0], F_true[~cinfo.feasible_mask, 1], c="red", marker="x", label="Infeasible")
plt.title("Manual Penalty Results (True Objectives)")
plt.legend()
plt.show()